# Hypothesis Testing
#### To statistically test whether the option expiry of Nifty50 causes the market to close on the downside 

In [ ]:
# Import necessary libraries
from dateutil.relativedelta import relativedelta
from datetime import date, timedelta
from scipy import stats
import yfinance as yf
import pandas as pd
import numpy as np

In [ ]:

# Define the ticker symbol for Nifty 50
ticker_symbol = '^NSEI'

# Define the start and end dates for the data
end_date = date.today()  # Set the end date as today's date
start_date = end_date - relativedelta(years=3)  # Set the start date as 2 years before the end date

# Download historical data
df = yf.download(ticker_symbol, start=start_date, end=end_date)

In [ ]:
# Reset the index of the DataFrame to the default integer index and modify the DataFrame in place
df.reset_index(inplace=True)

# Create a new column 'Day of Week' that contains the name of the day (e.g., 'Monday', 'Tuesday') from the 'Date' column
df['Day of Week'] = df['Date'].dt.day_name()

# Create a new column 'No of WeekDay' that contains the day of the week as a number (0=Monday, 6=Sunday)
df['No of WeekDay'] = df['Date'].dt.weekday

# Calculate the percentage change in the 'Close' column from the previous day and create a new column '%_CloseChange'
df['%_CloseChange'] = df['Close'].pct_change()

# Create a new column 'Market_Direction' to indicate whether the market moved up or down based on '%_CloseChange'
df['Market_Direction'] = np.where(df['%_CloseChange'] > 0, 'UP', 'DOWN')

# create the column which will divide the data into expiry days and non expiry days
df['Is_Expiry'] = np.where(df['Day of Week'] =='Thursday' , 'Expiry', 'Not Expiry')

In [ ]:
# Separate the data into two groups
expiry_returns = df[df['Is_Expiry'] == 'Expiry']['%_CloseChange'].dropna()
non_expiry_returns = df[df['Is_Expiry'] == 'Not Expiry']['%_CloseChange'].dropna()

In [ ]:
# Perform a t-test
t_stat, p_value = stats.ttest_ind(expiry_returns, non_expiry_returns, alternative='less')

# Conclusion
if p_value < 0.05:
    print(f"T-statistic: {t_stat}, P-value: {p_value}")
    print("Reject the null hypothesis. Option expiry days have a significantly lower return.")
else:
    print(f"T-statistic: {t_stat}, P-value: {p_value}")
    print("Fail to reject the null hypothesis. No significant difference in returns.")


In [ ]:
df.to_csv('nifty_data.csv')